In [1]:
!pip install trueskill

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Processing /home/jovyan/work/andrew.zhuang/.cache/pip/wheels/cf/d3/d9/9d28dcef9c00d7e26f03c712bdc233db93298cbc2a9c3d4d18/trueskill-0.4.5-py2-none-any.whl
You should consider upgrading via the '/artifacts/virtualenv/python2/bin/python -m pip install --upgrade pip' command.


In [2]:
from __future__ import division


def expected(A, B):
    """
    Calculate expected score of A in a match against B
    :param A: Elo rating for player A
    :param B: Elo rating for player B
    """
    return 1 / (1 + 10 ** ((B - A) / 400))


def elo(old, exp, score, k=32):
    """
    Calculate the new Elo rating for a player
    :param old: The previous Elo rating
    :param exp: The expected score for this match
    :param score: The actual score for this match
    :param k: The k-factor for Elo (default: 32)
    """
    return old + k * (score - exp)

In [3]:
import requests
import json
import trueskill as ts
from tqdm import tqdm
import datetime

In [4]:
def baddy_api_data(ep_string):
    api_url = "https://baddie-match.herokuapp.com/api/" + str(ep_string)
    response = requests.get(api_url)
    return (response.json())

In [5]:
matches = baddy_api_data('matches')
players = baddy_api_data('players')

In [6]:
from operator import itemgetter
# get recent matches
matches = [match for match in matches if datetime.datetime.strptime(match['date_added'], '%Y-%m-%d-%H:%M:%S') > datetime.datetime(2022, 9, 1)]
# get sorted matches
matches = sorted(matches, key=itemgetter('date_added'), reverse=False)

In [7]:
def initialize_player_stat_dict(players_in):
    # dictionary with player id as key
    # store list of [name, singles elo rating, doubles elo rating, singles games played, singles wins, singles losses,
    # doubles games played, doubles wins, doubles losses, list of singles score deltas, list of doubles score deltas, singles elo 2] 

    player_stats = {}
    for p in players:
        player_stats[p['id']] = [p['first_name'] + ' ' + p['last_name'], ts.Rating(), ts.Rating(),0,0,0,0,0,0,[],[],1000]
    
    return (player_stats)

player_stats = initialize_player_stat_dict(players)

In [8]:
def update_match(match, player_stats):
    if (match['score'][0] > 0 or match['score'][1] > 0):
        updated_stats = update_game(match, 0, 1, player_stats)
    
    if (match['score'][2] > 0 or match['score'][3] > 0):
        updated_stats = update_game(match, 2, 3, updated_stats)
        
    if (match['score'][4] > 0 or match['score'][5] > 0):
        updated_stats = update_game(match, 4, 5, updated_stats)
    
    return (updated_stats)

In [9]:
def update_game(match, score_1_pos, score_2_pos, player_stats):
    # get deltas
    delta_t1 = match['score'][score_1_pos] - match['score'][score_2_pos]
    delta_t2 = match['score'][score_2_pos] - match['score'][score_1_pos]

    # determine winner (0 flags the winner)
    if delta_t1 > 0:
        result = [0, delta_t1]
        win_loss = [0, 1]
    else:
        result = [-delta_t1, 0]
        win_loss = [1, 0]

    # check singles or doubles
    if match['event'] != 'Singles':
        # Doubles update

        # list of elo scores for team 
        p1_elo = player_stats[match['players'][0]][2]
        p2_elo = player_stats[match['players'][1]][2]

        p3_elo = player_stats[match['players'][2]][2]
        p4_elo = player_stats[match['players'][3]][2]

        t1 = [p1_elo, p2_elo]
        t2 = [p3_elo, p4_elo]

        # new elo
        new_elo = ts.rate([t1, t2], ranks=result)

        # set new elos
        player_stats[match['players'][0]][2] = new_elo[0][0]
        player_stats[match['players'][1]][2] = new_elo[0][1]
        player_stats[match['players'][2]][2] = new_elo[1][0]
        player_stats[match['players'][3]][2] = new_elo[1][1]

        # games played
        player_stats[match['players'][0]][6] += 1
        player_stats[match['players'][1]][6] += 1
        player_stats[match['players'][2]][6] += 1
        player_stats[match['players'][3]][6] += 1

        # wins
        player_stats[match['players'][0]][7] += abs(win_loss[1])
        player_stats[match['players'][1]][7] += abs(win_loss[1])
        player_stats[match['players'][2]][7] += abs(win_loss[0])
        player_stats[match['players'][3]][7] += abs(win_loss[0])

        # losses
        player_stats[match['players'][0]][8] += abs(win_loss[0])
        player_stats[match['players'][1]][8] += abs(win_loss[0])
        player_stats[match['players'][2]][8] += abs(win_loss[1])
        player_stats[match['players'][3]][8] += abs(win_loss[1])

        # deltas
        player_stats[match['players'][0]][10].append(delta_t1)
        player_stats[match['players'][1]][10].append(delta_t1)
        player_stats[match['players'][2]][10].append(delta_t2)
        player_stats[match['players'][3]][10].append(delta_t2)

        return (player_stats)

    else:
        # Singles update

        # list of elo scores for team 
        p1_elo = player_stats[match['players'][0]][2]
        p2_elo = player_stats[match['players'][1]][2]

        t1 = [p1_elo]
        t2 = [p2_elo]

        # new elo
        new_elo = ts.rate([t1, t2], ranks=result)
        
        p1_elo2 = elo(player_stats[match['players'][0]][11], expected(player_stats[match['players'][0]][11], player_stats[match['players'][1]][11]), delta_t1/2)
        p2_elo2 = elo(player_stats[match['players'][1]][11], expected(player_stats[match['players'][1]][11], player_stats[match['players'][0]][11]), delta_t2/2)
                                                                      
        # set new elos
        player_stats[match['players'][0]][1] = new_elo[0][0]
        player_stats[match['players'][1]][1] = new_elo[1][0]

        player_stats[match['players'][0]][11] = p1_elo2
        player_stats[match['players'][1]][11] = p2_elo2
                                                                      
        # games played
        player_stats[match['players'][0]][3] += 1
        player_stats[match['players'][1]][3] += 1

        # wins
        player_stats[match['players'][0]][4] += abs(win_loss[1])
        player_stats[match['players'][1]][4] += abs(win_loss[0])

        # losses
        player_stats[match['players'][0]][5] += abs(win_loss[0])
        player_stats[match['players'][1]][5] += abs(win_loss[1])

        # deltas
        player_stats[match['players'][0]][9].append(delta_t1)
        player_stats[match['players'][1]][9].append(delta_t2)

        return (player_stats)
    

In [10]:
for m in tqdm(matches):
    player_stats = update_match(m, player_stats)

100%|██████████| 120/120 [00:00<00:00, 810.08it/s]


In [11]:
stat_columns = ['name', 'singles rating', 'doubles rating', 'singles games played', 'singles wins', 'singles losses',
                'doubles games played', 'doubles wins', 'doubles losses', 'singles score deltas', 'list of doubles score deltas', 'singles elo']

df_player_stats = pd.DataFrame.from_dict(player_stats, orient='index', dtype=None, columns=stat_columns)

In [19]:
df_doubles_ranks = df_player_stats[df_player_stats['doubles games played'] >= 1]
df_doubles_ranks['doubles win pct'] =  df_doubles_ranks['doubles wins']/df_doubles_ranks['doubles games played']
df_doubles_ranks = df_doubles_ranks.sort_values(by=['doubles rating'], ascending = False)

df_singles_ranks = df_player_stats[df_player_stats['singles games played'] >= 1]
df_singles_ranks['singles win pct'] =  df_singles_ranks['singles wins']/df_singles_ranks['singles games played']
df_singles_ranks = df_singles_ranks.sort_values(by=['singles elo'], ascending = False)

/artifacts/virtualenv/python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/artifacts/virtualenv/python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
df_doubles_ranks[['name', 'doubles rating', 'doubles games played', 'doubles wins', 'doubles losses', 'doubles win pct']]

,name,doubles rating,doubles games played,doubles wins,doubles losses,doubles win pct
57,Andrew Zhuang,"trueskill.Rating(mu=34.114, sigma=3.191)",18,13,5,0.722222
55,Kevin Wang,"trueskill.Rating(mu=31.519, sigma=4.596)",10,8,2,0.800000
51,Jacob Kim,"trueskill.Rating(mu=30.265, sigma=2.556)",30,18,12,0.600000
61,Woon K,"trueskill.Rating(mu=27.185, sigma=5.750)",4,3,1,0.750000
38,Jenny Lei,"trueskill.Rating(mu=27.122, sigma=3.971)",13,8,5,0.615385
54,Kyle To,"trueskill.Rating(mu=27.035, sigma=2.294)",43,22,21,0.511628
58,Maggie Wong,"trueskill.Rating(mu=26.761, sigma=2.691)",33,21,12,0.636364
59,Jessica Cheng,"trueskill.Rating(mu=26.261, sigma=3.283)",17,12,5,0.705882
62,Jackie Dent,"trueskill.Rating(mu=26.037, sigma=6.508)",2,1,1,0.500000
64,Minh Cheng,"trueskill.Rating(mu=25.692, sigma=4.498)",7,4,3,0.571429


In [21]:
df_singles_ranks[['name', 'singles elo', 'singles rating', 'singles games played', 'singles wins', 'singles losses', 'singles win pct']]

,name,singles elo,singles rating,singles games played,singles wins,singles losses,singles win pct
55,Kevin Wang,1407.756376,"trueskill.Rating(mu=32.170, sigma=4.334)",9,7,2,0.777778
39,Karla Mccallum,1338.979777,"trueskill.Rating(mu=27.296, sigma=3.721)",20,12,8,0.600000
61,Woon K,1318.006592,"trueskill.Rating(mu=30.317, sigma=6.706)",2,2,0,1.000000
57,Andrew Zhuang,1194.366113,"trueskill.Rating(mu=32.720, sigma=3.078)",6,4,2,0.666667
46,Clement Chow,1100.118074,"trueskill.Rating(mu=34.499, sigma=6.006)",5,4,1,0.800000
64,Minh Cheng,1098.266207,"trueskill.Rating(mu=22.466, sigma=3.919)",3,2,1,0.666667
44,Ivan Cheng,1087.723549,"trueskill.Rating(mu=25.329, sigma=3.683)",8,3,5,0.375000
45,Darren Choi,915.778616,"trueskill.Rating(mu=22.429, sigma=2.789)",11,8,3,0.727273
62,Jackie Dent,891.569163,"trueskill.Rating(mu=20.166, sigma=6.732)",2,0,2,0.000000
36,Siya Lai,888.307466,"trueskill.Rating(mu=21.422, sigma=3.258)",3,2,1,0.666667


In [19]:
matches[0]

{u'category': u'practice',
 u'date_added': u'2022-09-06-00:00:00',
 u'event': u'Singles',
 u'id': 62,
 u'last_edit': u'2022-09-06-00:00:00',
 u'players': [56, 49],
 u'score': [21, 13, 0, 0, 0, 0],
 u'winners': [56]}

In [85]:
r1 = ts.Rating()  # 1P's skill
r2 = ts.Rating()  # 2P's skill
r3 = ts.Rating()  # 3P's skill
t1 = [r1]  # Team A contains just 1P
t2 = [r2]  # Team B contains 2P and 3P

(new_r1,), (new_r2,) = ts.rate([t1, t2], ranks=[-1, 0])
print(new_r1)
print(new_r2)


trueskill.Rating(mu=29.396, sigma=7.171)
trueskill.Rating(mu=20.604, sigma=7.171)


In [66]:
ts.rate_1vs1(r1, r2)

(trueskill.Rating(mu=29.396, sigma=7.171),
 trueskill.Rating(mu=20.604, sigma=7.171))

In [77]:
r1 = ts.Rating()  # 1P's skill
r2 = ts.Rating()  # 2P's skill
r3 = ts.Rating()  # 3P's skill
r4 = ts.Rating()
t1 = [r1, r2]  # Team A contains just 1P
t2 = [r3, r4]  # Team B contains 2P and 3P

new_elos = ts.rate([t1, t2], ranks=[0, 1])

In [338]:
new_elos

[(trueskill.Rating(mu=28.108, sigma=7.774),
  trueskill.Rating(mu=28.108, sigma=7.774)),
 (trueskill.Rating(mu=21.892, sigma=7.774),
  trueskill.Rating(mu=21.892, sigma=7.774))]

In [5]:
import pandas as pd
df_matches = pd.DataFrame(matches)
df_matches = df_matches.sort_values(by=['date_added'])

In [6]:
df_matches = pd.DataFrame(matches)

In [32]:
df_matches = df_matches.sort_values(by=['date_added'])

In [ ]:
for i in range(len(df_matches)):

In [164]:
df_matches[df_matches['event'] == 'Singles']

,category,date_added,event,id,last_edit,players,score,winners
3,practice,2022-02-15-00:00:00,Singles,38,2022-02-15-00:00:00,"[46, 56]","[21, 11, 0, 0, 0, 0]",[46]
4,practice,2022-02-15-00:00:00,Singles,39,2022-02-15-00:00:00,"[56, 39]","[21, 13, 0, 0, 0, 0]",[56]
5,practice,2022-02-15-00:00:00,Singles,40,2022-02-15-00:00:00,"[56, 38]","[21, 16, 0, 0, 0, 0]",[56]
6,practice,2022-02-15-00:00:00,Singles,41,2022-02-15-00:00:00,"[40, 53]","[9, 21, 0, 0, 0, 0]",[53]
7,practice,2022-02-17-00:00:00,Singles,51,2022-02-17-00:00:00,"[56, 46]","[21, 18, 0, 0, 0, 0]",[56]
0,practice,2022-09-06-00:00:00,Singles,62,2022-09-06-00:00:00,"[56, 49]","[21, 13, 0, 0, 0, 0]",[56]
30,practice,2022-09-09-00:00:00,Singles,72,2022-09-09-22:00:00,"[38, 39]","[21, 13, 0, 0, 0, 0]",[38]
31,practice,2022-09-09-00:00:00,Singles,73,2022-09-09-22:00:00,"[49, 42]","[21, 3, 0, 0, 0, 0]",[49]
36,practice,2022-09-09-00:00:00,Singles,78,2022-09-09-22:00:00,"[55, 51]","[11, 9, 0, 0, 0, 0]",[55]
33,practice,2022-09-09-00:00:00,Singles,75,2022-09-09-22:00:00,"[49, 51]","[21, 18, 0, 0, 0, 0]",[49]


In [329]:
df_doubles_ranks = df_doubles_ranks[df_doubles_ranks['doubles games played'] > 10]
df_doubles_ranks['doubles win pct'] =  df_doubles_ranks['doubles wins']/df_doubles_ranks['doubles games played']

In [330]:
df_doubles_ranks

,name,singles rating,doubles rating,singles games played,singles wins,singles losses,doubles games played,doubles wins,doubles losses,singles score deltas,list of doubles score deltas,doubles win pct
57,Andrew Zhuang,"trueskill.Rating(mu=32.720, sigma=3.078)","trueskill.Rating(mu=34.114, sigma=3.191)",6,4,2,18,13,5,"[8, 7, 14, -10, 5, -5]","[10, 5, 2, 9, -3, 4, -7, 5, 9, -8, -2, 10, 5, 5, -7, 5, 6, 4]",0.722222
51,Jacob Kim,"trueskill.Rating(mu=23.596, sigma=4.758)","trueskill.Rating(mu=30.265, sigma=2.556)",4,0,4,30,18,12,"[-3, -2, -11, -12]","[7, -2, 5, 2, -5, -6, 4, -2, 12, 2, 2, -9, 3, 2, 2, -2, 2, -4, 6, 4, -9, 4, 8, 2, -10, -5, -2, 7, -5, 7]",0.600000
38,Jenny Lei,"trueskill.Rating(mu=23.415, sigma=3.918)","trueskill.Rating(mu=27.122, sigma=3.971)",17,9,8,13,8,5,"[8, -2, 8, -7, -7, 3, 5, 8, 12, -3, 5, -8, -5, 6, 7, -4, -10]","[-2, 7, 9, 3, -3, 2, 2, 3, -2, -5, -4, 3, 2]",0.615385
54,Kyle To,"trueskill.Rating(mu=23.904, sigma=3.539)","trueskill.Rating(mu=27.035, sigma=2.294)",5,2,3,43,22,21,"[-7, 5, 8, -8, -7]","[2, 5, 21, -2, -2, -5, -2, 5, 4, 9, 11, 6, -2, 11, -10, -5, -2, -2, -2, -2, -2, 4, 5, -7, 2, -2, -4, 6, 13, -5, 9, -4, 8, 2, 10, 5, 13, 9, 2, -7, ...",0.511628
58,Maggie Wong,"trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=26.761, sigma=2.691)",0,0,0,33,21,12,[],"[5, 2, -7, -6, 4, -9, -7, 6, 11, 6, -3, 3, 2, 2, 3, 11, -9, 3, -3, 2, 2, 2, 5, -7, -4, -13, -3, 2, -7, 11, 2, 5, 6]",0.636364
59,Jessica Cheng,"trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=26.261, sigma=3.283)",0,0,0,17,12,5,[],"[5, -8, 2, 6, 10, 2, 3, 9, -3, 3, -8, 13, 9, -2, 7, 7, -7]",0.705882
47,Thomas Dent,"trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=25.067, sigma=2.394)",0,0,0,37,13,24,[],"[-2, -5, -21, 9, -2, 5, -2, 2, -10, -2, 6, -4, -2, 12, 10, -2, -2, -4, 7, -2, -2, -5, 7, 2, -2, 7, 4, -13, 5, -10, -5, -13, -9, -6, -6, -7, 7]",0.351351
44,Ivan Cheng,"trueskill.Rating(mu=25.329, sigma=3.683)","trueskill.Rating(mu=24.448, sigma=3.889)",8,3,5,16,7,9,"[-6, -5, -2, 18, -6, -2, 2, 13]","[-7, 2, 2, 2, -8, 2, -2, 11, 2, -12, -5, -2, -2, -3, -10, 5]",0.437500
39,Karla Mccallum,"trueskill.Rating(mu=27.296, sigma=3.721)","trueskill.Rating(mu=23.973, sigma=3.847)",20,12,8,13,7,6,"[-10, -8, 2, -8, 7, 7, -3, 7, -5, -8, 8, 5, 9, 6, -3, -4, 11, 10, 4, 10]","[-5, -8, -5, 7, 3, -3, -3, 2, 2, 2, 13, 3, -6]",0.538462
36,Siya Lai,"trueskill.Rating(mu=21.422, sigma=3.258)","trueskill.Rating(mu=23.708, sigma=2.936)",3,2,1,25,13,12,"[-12, 3, 4]","[8, 5, 2, -7, 6, -6, 4, -2, -2, -3, -11, 3, -2, -2, -2, -5, 7, 4, 13, 3, -11, -2, 6, 3, 2]",0.520000


In [339]:
a = 1000
b = 1000

expected(a, b)

0.5

In [342]:
elo(a, expected(a, b), -5)

824.0

In [337]:
from __future__ import division


def expected(A, B):
    """
    Calculate expected score of A in a match against B
    :param A: Elo rating for player A
    :param B: Elo rating for player B
    """
    return 1 / (1 + 10 ** ((B - A) / 400))


def elo(old, exp, score, k=32):
    """
    Calculate the new Elo rating for a player
    :param old: The previous Elo rating
    :param exp: The expected score for this match
    :param score: The actual score for this match
    :param k: The k-factor for Elo (default: 32)
    """
    return old + k * (score - exp)